PART A:- Check if 4-digit PIN is commonly used

In [52]:

COMMONLY_USED_MPINS_4 = {
    "1234", "0000", "1111", "2222", "3333", "4444",
    "5555", "6666", "7777", "8888", "9999", "1122", "1212", "1313","6969","4321","1004","2000","2580","0987","1010","4321"
}

COMMONLY_USED_MPINS_6 = {
 "123456", "000000", "111111", "654321", "121212",
    "112233", "999999", "159753", "123123", "789456",
    "101010", "123321", "456789", "121314", "111222",
    "314159", "777777", "010101"
}

In [53]:
#here we check the commonly used mpins of 4 and 6 digit
def is_commonly_used(mpin):
    if len(mpin)==4:
        return mpin in COMMONLY_USED_MPINS_4
    elif len(mpin)==6:
        return mpin in COMMONLY_USED_MPINS_6
    return False



    

Extract all position combination from a Date

PART B:-Check strength (WEAK/STRONG) using demographics & common PINs

In [54]:
# for 4-digit
#first possible combination is day+month .ex=02(day)04(month)-0204
#2nd possible combination is just reverse first-0402
#3rd possible combination is year_short +day .ex ;like if someone birth year is 1998 then year_short is 98 if day is 11.then mpin is=9811
#4th possible is just reverse third =1198.
#5th is year_short + month .ex =9812(where birth year =1998 and month is 12th i.e december.
#6th is just reverse 5th .ex.1298.
#7th someone may be use repeat of year_short+year_short.like 9898.
#8th repeat of day +day.like 0909
#9th repeat of month+month.like 0808
#10th may be lasttwo digit of year reverse also possible.like birth year is 1998 and day is 13.then combination is :-8913
#11th just reverse of 10th

In [70]:
def extract_date_combinations(date, pin_length=None):
    if not date or len(date.split('-')) != 3:
        return []

    day, month, year = date.split('-')
    year_short = year[-2:]
    year_short_rev = year_short[::-1]

    combinations = set()

    four_digit = {
        day + month,
        month + day,
        year_short + day,
        day + year_short,
        year_short + month,
        month + year_short,
        year_short + year_short,
        day + day,
        month + month,
        year_short_rev + day,
        day + year_short_rev,
    }

    six_digit = {
        day + month + year_short,
        day + year_short + month,
        month + day + year_short,
        month + year_short + day,
        year_short + day + month,
        year_short + month + day,
        year_short + year_short + day,
        day + day + year_short,
        day + month + year_short_rev,
        month + month + year_short,
    }

 
    if pin_length == 4:
        combinations.update(four_digit)
    elif pin_length == 6:
        combinations.update(six_digit)
    else:
        combinations.update(four_digit)
        combinations.update(six_digit)

    return list(combinations)


In [71]:
def is_common_pin(pin):
    if len(pin) == 4:
        return pin in COMMONLY_USED_MPINS_4
    elif len(pin) == 6:
        return pin in COMMONLY_USED_MPINS_6
    return False


In [72]:
def check_strength(pin,DOB=None,Spouse_DOB=None,Wedding_Anniversary=None):

    if len(pin) not in [4,6]:
        return "invalid_length",[]
    combos_func = extract_date_combinations(pin)
    if DOB and pin in combos_func(DOB):
        return "WEAK"
    if Spouse_DOB and pin in combos_func(Spouse_DOB):
        return "WEAK"
    if Wedding_Anniversary and pin in combos_func(Wedding_Anniversary):
        return "WEAK"
    return "STRONG"

PART-C  Strength Check with Reasons (4/6-digit) 

In [73]:
def check_strength_reason(pin, DOB, Spouse_DOB, Wedding_Anniversary):
    reasons = []
    if len(pin) not in [4,6]:
        return "INVALID_LENGTH", []
    if is_common_pin(pin):
        reasons.append("COMMONLY_USED")

    if DOB:
        combos = extract_date_combinations(DOB)
        if pin in combos:
            reasons.append("DEMOGRAPHIC_DOB_SELF")

    if Spouse_DOB:
        combos = extract_date_combinations(Spouse_DOB)
        if pin in combos:
            reasons.append("DEMOGRAPHIC_DOB_SPOUSE")

    if Wedding_Anniversary:
        combos = extract_date_combinations(Wedding_Anniversary)
        if pin in combos:
            reasons.append("DEMOGRAPHIC_ANNIVERSARY")

    if reasons:
        return "WEAK", reasons
    return "STRONG", []


PART D is Already integrated in PART A,B,C 

PART E :- TEST CASES

In [81]:
   def run_tests_cases():
    tests = [
        # for common used 4-digit
        ('0000', None, None, None, "WEAK", ["COMMONLY_USED"]),
        ('2580', None, None, None, "WEAK", ["COMMONLY_USED"]),
        ('4321', None, None, None, "WEAK", ["COMMONLY_USED"]),
        # for common used 6-digit
        ('000000', None, None, None, "WEAK", ["COMMONLY_USED"]),
        ('010101', None, None, None, "WEAK", ["COMMONLY_USED"]),
        ('112233', None, None, None, "WEAK", ["COMMONLY_USED"]),
        # DOB Match 4-digit
        ("0201", "02-01-1998", None, None, "WEAK", ["DEMOGRAPHIC_DOB_SELF"]),
        ("1004", "04-10-2000", None, None, "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SELF"]),
        ("1212", "12-12-2012", None, None, "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SELF"]),
        # DOB match 6-digit
        ("020198", "02-01-1998", None, None, "WEAK", ["DEMOGRAPHIC_DOB_SELF"]),
        ("101290", None, "10-12-1990", None, "WEAK", ["DEMOGRAPHIC_DOB_SPOUSE"]),
        ("150815", None, None, "15-08-2015", "WEAK", ["DEMOGRAPHIC_ANNIVERSARY"]),
        # Mixed common + demographic
        ("1111", "11-11-2011", None, None, "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SELF"]),
        ("888888", None, None, "08-08-1988", "WEAK", ["COMMONLY_USED", "DEMOGRAPHIC_ANNIVERSARY"]),
        # Invalid lengths
        ("12", None, None, None, "INVALID_LENGTH", []),
        ("123", None, None, None, "INVALID_LENGTH", []),
        ("12345", None, None, None, "INVALID_LENGTH", []),
        ("1234567", None, None, None, "INVALID_LENGTH", []),
        # Strong MPINs (random, not demographic-related or common)
        ("8392", "01-02-1990", "03-04-1991", "05-06-1992", "STRONG", []),
        ("839201", "01-02-1990", "03-04-1991", "05-06-1992", "STRONG", []),
        ("7594", None, None, None, "STRONG", []),
        ("930284", None, None, None, "STRONG", [])
    ]

    for i, (pin, DOB, Spouse_DOB, Wedding_Anniversary, expected_strength, expected_reasons) in enumerate(tests, 1):
        strength, reasons = check_strength_reason(pin, DOB, Spouse_DOB, Wedding_Anniversary)
        strength_match = (strength == expected_strength)
        reasons_match = (sorted(reasons) == sorted(expected_reasons))
        passed = strength_match and reasons_match
        print(f"Test {i}: PIN={pin}, Strength={strength}, Reasons={reasons} -> {'Passed ✅' if passed else 'Failed ❌'}")



In [82]:
if __name__ == "__main__":
    run_tests_cases()


Test 1: PIN=0000, Strength=WEAK, Reasons=['COMMONLY_USED'] -> Passed ✅
Test 2: PIN=2580, Strength=WEAK, Reasons=['COMMONLY_USED'] -> Passed ✅
Test 3: PIN=4321, Strength=WEAK, Reasons=['COMMONLY_USED'] -> Passed ✅
Test 4: PIN=000000, Strength=WEAK, Reasons=['COMMONLY_USED'] -> Passed ✅
Test 5: PIN=010101, Strength=WEAK, Reasons=['COMMONLY_USED'] -> Passed ✅
Test 6: PIN=112233, Strength=WEAK, Reasons=['COMMONLY_USED'] -> Passed ✅
Test 7: PIN=0201, Strength=WEAK, Reasons=['DEMOGRAPHIC_DOB_SELF'] -> Passed ✅
Test 8: PIN=1004, Strength=WEAK, Reasons=['COMMONLY_USED', 'DEMOGRAPHIC_DOB_SELF'] -> Passed ✅
Test 9: PIN=1212, Strength=WEAK, Reasons=['COMMONLY_USED', 'DEMOGRAPHIC_DOB_SELF'] -> Passed ✅
Test 10: PIN=020198, Strength=WEAK, Reasons=['DEMOGRAPHIC_DOB_SELF'] -> Passed ✅
Test 11: PIN=101290, Strength=WEAK, Reasons=['DEMOGRAPHIC_DOB_SPOUSE'] -> Passed ✅
Test 12: PIN=150815, Strength=WEAK, Reasons=['DEMOGRAPHIC_ANNIVERSARY'] -> Passed ✅
Test 13: PIN=1111, Strength=WEAK, Reasons=['COMMONLY

THANK YOU